# Summarize documents

(106-dotnet-retrieve-synthetics)[https://github.com/microsoft/kernel-memory/blob/main/examples/106-dotnet-retrieve-synthetics/Program.cs]

In [1]:
#r "nuget: Microsoft.KernelMemory.Core, 0.71.240820.1"

Installed Packages Microsoft.KernelMemory.Core, 0.71.240820.1

In [3]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
Console.WriteLine($"Use Azure Open AI : {useAzureOpenAI}");
Console.WriteLine($"Model : {model}");
Console.WriteLine($"Azure EndPoint : {azureEndpoint}");


Use Azure Open AI : True
Model : gpt-4o
Azure EndPoint : https://oai-ygo74-switzerland.openai.azure.com/


In [7]:
using Microsoft.KernelMemory;

// Using OpenAI for embeddings
var configEmbedding = new AzureOpenAIConfig {
    APIKey=apiKey,
    Endpoint=azureEndpoint,
    Auth=Microsoft.KernelMemory.AzureOpenAIConfig.AuthTypes.APIKey,
    Deployment="text-embedding-ada-002"
};

// Using LM Studio for text generation
var ollamaConfig = new OpenAIConfig
{
    Endpoint = "http://localhost:11434/v1/",
    TextModel = "phi3",
    TextModelMaxTokenTotal = 4096,
    APIKey = "lm-studio"
};

var config = new AzureOpenAIConfig {
    APIKey=apiKey,
    Endpoint=azureEndpoint,
    Auth=Microsoft.KernelMemory.AzureOpenAIConfig.AuthTypes.APIKey,
    Deployment=model
};

var memory = new KernelMemoryBuilder()
    .WithAzureOpenAITextEmbeddingGeneration(configEmbedding) // OpenAI
    // .WithOpenAITextGeneration(ollamaConfig) // LM Studio
    .WithAzureOpenAITextGeneration(config)
    .Build();


In [11]:
using Microsoft.KernelMemory.Context;

var context = new RequestContext();
context.SetArg("custom_summary_prompt_str", "Super extra summarize, use French, use short sentences, no list, no new lines. The answer must be short. Content: {{$input}}. Summary: ");
context.SetArg("custom_summary_target_token_size_int", 15); // Try to generate a token no longer than 15 tokens
context.SetArg("custom_summary_overlapping_tokens_int", 0); // Disable overlapping tokens

In [12]:
// Import a couple of documents to summarize.
// Note that we're using a custom set of steps, asking the pipeline to just summarize the docs (ie skipping chunking)
await memory.ImportDocumentAsync(new Document("doc1")
        .AddFile(@"D:\OneDrive\Documents\Enfants\Noémie\université\Lyon\Logement\BAIL GOBERT.pdf"),
    steps: Constants.PipelineOnlySummary,
    context: context);

In [13]:
// Fetch the list of summaries. The API returns one summary for each file.
var results = await memory.SearchSummariesAsync(filter: MemoryFilters.ByDocument("doc1"));

// Print the summaries
foreach (var result in results)
{
    Console.WriteLine($"== {result.SourceName} summary ==\n{result.Partitions.First().Text}\n");
}


== BAIL GOBERT.pdf summary ==
Sylvie loue un meublé à Noémie à Lyon pour 630€ plus 50€ de charges. Début le 1er août 2022 pour un an. Dépôt de garantie de 1 260€. Résiliation automatique si non-paiement. Respect des règlements et entretien obligatoires. Logement décent fourni. Préavis d'un mois pour résiliation. Bail reconduit annuellement. Sous-location interdite sans accord. Honoraires partagés. Loyer révisable annuellement. Conciliation ou tribunal en cas de litige.

